In [ ]:
import os
import mne
from config import BASE_DIR
mne.set_log_level("error")

In [ ]:
sub = "SB00"
sub_dir = f"{BASE_DIR}/control_clean/{sub}"
filename = f"{sub}_196-head-ch_resample250_filtered_scored_bad-epochs"
bad_channels_path = f"{sub_dir}/{sub}_bad_channels.txt"

In [3]:
raw = mne.io.read_raw(f"{sub_dir}/{filename}.fif")

In [4]:
# Check for NREM2 annotations longer than 280 seconds
long_nrem2_count = 0
long_nrem2_durations = []

for i, (desc, duration) in enumerate(zip(raw.annotations.description, raw.annotations.duration)):
    if desc == 'NREM2' and duration >= 280:
        long_nrem2_count += 1
        long_nrem2_durations.append(duration)

print(f"NREM2 annotations longer than 280 seconds: {long_nrem2_count}")
if long_nrem2_count > 0:
    print(f"Durations: {[f'{d:.1f}s' for d in sorted(long_nrem2_durations)]}")
else:
    print("All NREM2 annotations are 280 seconds or shorter")

NREM2 annotations longer than 280 seconds: 15
Durations: ['300.0s', '330.0s', '420.0s', '450.0s', '480.0s', '480.0s', '570.0s', '570.0s', '720.0s', '780.0s', '840.0s', '900.0s', '1170.0s', '1470.0s', '2700.0s']


In [5]:
# Calculate BAD epochs percentage by TIME duration
bad_time = sum(duration for desc, duration in zip(raw.annotations.description, raw.annotations.duration) 
               if desc.startswith('BAD'))
total_recording_time = raw.times[-1]  # Total recording duration in seconds
bad_time_percentage = (bad_time / total_recording_time) * 100

print(f"BAD time: {bad_time:.1f}s / {total_recording_time:.1f}s ({bad_time_percentage:.1f}%)")

BAD time: 1111.8s / 24804.0s (4.5%)


Look at the channels and mark bad ones!

In [ ]:
# Set MNE configuration for light theme
mne.set_config('MNE_BROWSER_THEME', 'light')  # Set browser theme to light
mne.viz.set_browser_backend('qt')

# Alternative approach: set Qt style
os.environ['QT_STYLE_OVERRIDE'] = 'Fusion'  # Use Fusion style which is lighter

raw.plot(duration=40)

In [7]:
def sort_electrode_names(electrode_list):
    return sorted(electrode_list, key=lambda x: int(x[1:]) if x.startswith('E') and x[1:].isdigit() else float('inf'))

In [8]:
original_bad_channels = sort_electrode_names([str(x) for x in raw.info['bads']])
bad_percentage = (len(original_bad_channels) / len(raw.ch_names)) * 100
print(f"Bad channels marked: {original_bad_channels}")
print(f"Count: {len(original_bad_channels)} ({bad_percentage:.2f}%)")

Bad channels marked: ['E149', 'E158']
Count: 2 (1.02%)


In [9]:
with open(bad_channels_path, 'w') as f:
    for ch in original_bad_channels:
        f.write(f"{ch}\n")

In [10]:
# Load bad channels from file and set them as bad
if os.path.exists(bad_channels_path):
    with open(bad_channels_path, 'r') as f:
        loaded_bad_channels = [line.strip() for line in f if line.strip()]
    
    raw.info['bads'] = loaded_bad_channels
    print(f"Loaded and set {len(loaded_bad_channels)} bad channels from file:")
    print(f"Bad channels: {sort_electrode_names(loaded_bad_channels)}")
else:
    print(f"Bad channels file not found: {bad_channels_path}")

Loaded and set 3 bad channels from file:
Bad channels: ['E75', 'E82', 'E144']


In [10]:
raw.load_data()
raw.interpolate_bads()
print(f"Channels interpolated.")

Channels interpolated.


In [11]:
raw.set_eeg_reference('average', projection=False, ch_type='eeg')
print(f"Re-referenced to average")

Re-referenced to average


In [ ]:
raw.plot(duration=40)

In [12]:
output_path = f"{filename}_bad-channels_avg-ref"
raw.save(f"{sub_dir}/{output_path}.fif")